In [20]:
# Create Naive Bayes classifier using TD-IDF vectorizer and pickle results. This model can be 
# trained on job resumes or Indeed job postings. Steps include:
# 1. Set the model parameters
# 2. Getting a combined list of salaries and only using job titles with 500+ salary records
# 3. Getting a list of resumes using this list of job titles and remove any job titles with 
#    less than 500 resumes
# 4. Run TD-IDF vectorizer and Naive Bayes model training
# 5. Test the model using "List Most Relevant Skills"
# 6. Test the model using "Document Similarity Score"

In [21]:
import datetime
import math
import numpy as np
import pandas as pd
import pickle
from itertools import product
from random import shuffle
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from imblearn.over_sampling import SMOTE


# Custom function in functions folder
from functions.word_preprocessing import *

pd.set_option('display.max_colwidth', 500)

In [2]:
directory = '/Users/kwheatley/Desktop/Capstone/gcloud_data/'

# Model Parameters

In [3]:
parameters = {
                "doc_type":"indeed_resume", # Use "indeed_resume" or "indeed_postings"
                "min_salary_records":100, # Filter out all jobs with less than specified salary records
                "min_job_summaries":1000, # Filter out all jobs with less than specified job summaries
                "min_ngram":2, # For TD-IDF vectorizer
                "max_ngram":4, # For TD-IDF vectorizer
                "min_df":0, # For TD-IDF vectorizer, ignore features in less than this number of documents
                "train_test_split":0.05, # For train-test split
                "random_state":1, # For train-test split
                "alpha":0.1, # For Naive Bayes model
                "num_skills":50, # Number of skill to show per job 
}

# Load Salary Datasets

In [4]:
# Load the both salary datasets
salary1 = pd.read_csv(directory+'02_salaries_h1b.csv')
salary2 = pd.read_csv(directory+'02_salaries_greencard.csv')

# Combine salary datasets
temp_salary1 = salary1[['role','city','state','start_year','cleaned_job_title','experiences','salary']]
temp_salary2 = salary2[['job_title','city','state','decision_year','cleaned_job_title','experiences','salary_amount']]
temp_salary1.columns = ['original_role','city','state','start_year','cleaned_job_title','experiences','salary']
temp_salary2.columns = ['original_role','city','state','start_year','cleaned_job_title','experiences','salary']
combined_salaries = temp_salary1.append(temp_salary2)

# Remove salaries with null value and convert to int
combined_salaries = combined_salaries[~combined_salaries.salary.isnull()]
combined_salaries.salary = combined_salaries.salary.astype(int)

# Fill any NaN fields with no_value and convert each column into a list
combined_salaries = combined_salaries.fillna('no_value')
combined_salaries.experiences = combined_salaries.experiences.apply(lambda x: 
                                                list([item.strip() for item in x.split(',')]))
combined_salaries.original_role = combined_salaries.original_role.apply(lambda x: [x])
combined_salaries.city = combined_salaries.city.apply(lambda x: [x])
combined_salaries.state = combined_salaries.state.apply(lambda x: [x])
combined_salaries.start_year = combined_salaries.start_year.apply(lambda x: [x])
combined_salaries.cleaned_job_title = combined_salaries.cleaned_job_title.apply(lambda x: [x])
combined_salaries.salary = combined_salaries.salary.apply(lambda x: [x])

# Perform a pivot on the columns to split out rows with multiple experience level qualifiers
combined_salaries = pd.DataFrame([j for i in combined_salaries.values for j in product(*i)],
                                      columns = combined_salaries.columns)

# Get List of Jobs

In [5]:
# Choose all jobs with `min_salary_records` or more records
temp = combined_salaries.groupby(['cleaned_job_title']).count().salary.reset_index()
jobs_to_model = temp[temp.salary >= parameters['min_salary_records']]
pd.DataFrame(jobs_to_model.cleaned_job_title.unique()).to_csv(directory+'03_relevant_job_titles.csv',index=False)
combined_salaries = combined_salaries[combined_salaries.cleaned_job_title\
                                                       .fillna('').isin(jobs_to_model.cleaned_job_title)]
print("Number of jobs with "+str(parameters['min_salary_records'])+"+ salary records:", jobs_to_model.cleaned_job_title.count())

# combined_salaries.groupby(['cleaned_job_title'])\
# .salary.agg(['count','mean','min','max','median','std']).to_csv("sample.csv")

Number of jobs with 100+ salary records: 867


# Load Job Summaries

In [6]:
# Load resume data
if parameters['doc_type'] == 'indeed_resume':
    data = pd.read_csv(directory+'02_resumes_work.csv')
if parameters['doc_type'] == 'indeed_postings':
    data = pd.read_csv(directory+'02_job_posts_indeed.csv')

# Remove all null cleaned_job_title records
jobs_descriptions = data[~data.cleaned_job_title.isnull()]

In [7]:
# Filter to only jobs specified by the jobs_to_model list
jobs_descriptions = jobs_descriptions[jobs_descriptions.cleaned_job_title\
                                      .isin(jobs_to_model.cleaned_job_title)]

# Rename job summary field
if parameters['doc_type'] == 'indeed_resume':
    jobs_descriptions.rename(columns = {'descript':'summary_text'}, inplace=True)

In [8]:
# Print the number of records left after removing irrelevant jobs
print("Number of resume entries available:", 
      str(jobs_descriptions.cleaned_job_title.count()) +"/"+
      str(data.cleaned_job_title.count()))

Number of resume entries available: 983090/3105493


In [9]:
# Remove all jobs without `min_job_summaries` or more resume entries
cnt_resumes_available = jobs_descriptions.groupby('cleaned_job_title')\
                                .count().reset_index()
cnt_resumes_available = list(cnt_resumes_available[
            cnt_resumes_available.summary_text>parameters['min_job_summaries']].cleaned_job_title)
jobs_descriptions = jobs_descriptions[jobs_descriptions.cleaned_job_title\
                       .isin(cnt_resumes_available)]

# Save off list of resume ids
if parameters['doc_type'] == 'indeed_resume':
    # Save the list of resume ids for resumes being used
    pd.DataFrame(jobs_descriptions.resume_id.unique())\
                .to_csv(directory+'03_relevant_resume_ids.csv',index=False)
        
# Remove duplicate data
jobs_descriptions = jobs_descriptions[['cleaned_job_title','summary_text','from_year']].drop_duplicates()

print("Number of jobs with "+str(parameters['min_job_summaries'])+"+ resume entries:", len(cnt_resumes_available))
print("Number of resume entries available now:", jobs_descriptions.cleaned_job_title.count())

Number of jobs with 1000+ resume entries: 148
Number of resume entries available now: 759405


# Save Salary Data for App

In [10]:
# This code saves the cleaned salary information back to the main data folder
combined_salaries.to_csv(directory+'03_cleaned_salaries_for_app.csv',index=False)

# Data Preprocess Unbalanced Classes (Doc SimilarityModel)

In [11]:
# Down sample the first model
# SMOTE up sample the second model
# Run the model on different periods of time (just for software engineers)

In [12]:
max_number_records = 2500

# This code samples the number of records to remove excessive numbers
new_job_descriptions = pd.DataFrame()
for name, group in jobs_descriptions.groupby('cleaned_job_title'):
    if group[group.from_year < 2017].cleaned_job_title.count() > max_number_records:
        new_job_descriptions = pd.concat([new_job_descriptions,group[group.from_year < 2017]\
            .sort_values(by='from_year', ascending=False).head(max_number_records)])
    elif group[group.from_year < 2018].cleaned_job_title.count() > max_number_records:
        new_job_descriptions = pd.concat([new_job_descriptions,group[group.from_year < 2018]\
            .sort_values(by='from_year', ascending=False).head(max_number_records)])
    else:
        new_job_descriptions = pd.concat([new_job_descriptions,group\
            .sort_values(by='from_year', ascending=False).head(max_number_records)])

In [13]:
# from functions.word_preprocessing
x_data = preprocess_list(new_job_descriptions.summary_text)

# Create labels using cleaned_job_title
y_labels = new_job_descriptions.cleaned_job_title

In [16]:
# Split the data into test and train datasets
X_train, X_test, y_train, y_test = train_test_split(x_data, 
                                                    y_labels,
                                                    test_size=parameters['train_test_split'],
                                                    random_state=parameters['random_state'])

In [17]:
print("Start:", datetime.datetime.now())

# Train TF-IDF vectorizer model
vect = TfidfVectorizer(min_df=parameters['min_df'], 
                       ngram_range=(parameters['min_ngram'], parameters['max_ngram'])
                      ).fit(X_train)
X_train_vectorized = vect.transform(X_train)

print("End:", datetime.datetime.now())

print('Vocabulary len:', len(vect.get_feature_names()))

Start: 2018-07-18 16:09:17.317685
End: 2018-07-18 16:55:00.848302
Vocabulary len: 40581936


In [22]:
sm = SMOTE(kind='regular')
X_res, y_res = sm.fit_sample(X_train_vectorized, y_train)

In [24]:
temp_display = pd.DataFrame(y_res)
temp_display.columns = ['range']
temp_display['counter'] = 1
temp_display.groupby('range').count().reset_index()

,range,counter
0,account executive,2400
1,account manager,2400
2,accountant,2400
3,accounting,2400
4,accounting clerk,2400
5,accounting manager,2400
6,administrative assistant,2400
7,analyst,2400
8,android engineer,2400
9,applications engineer,2400


# Data Preprocess Unbalanced Classes (List Most Relevant Skills)

In [ ]:
# max_number_records = 10000

# # This code samples the number of records to remove excessive numbers
# new_job_descriptions = pd.DataFrame()
# for name, group in jobs_descriptions.groupby('cleaned_job_title'):
#     if group[group.from_year < 2017].cleaned_job_title.count() > max_number_records:
#         new_job_descriptions = pd.concat([new_job_descriptions,group[group.from_year < 2017]\
#             .sort_values(by='from_year', ascending=False).head(max_number_records)])
#     elif group[group.from_year < 2018].cleaned_job_title.count() > max_number_records:
#         new_job_descriptions = pd.concat([new_job_descriptions,group[group.from_year < 2018]\
#             .sort_values(by='from_year', ascending=False).head(max_number_records)])
#     else:
#         new_job_descriptions = pd.concat([new_job_descriptions,group\
#             .sort_values(by='from_year', ascending=False).head(max_number_records)])

# Train Model

In [ ]:
# Train Multinomial Naive Bayes model
model = MultinomialNB(alpha=parameters['alpha'])
model.fit(X_res, y_res)

y_pred = model.predict(vect.transform(X_test))
print('Accuracy: %.2f%%' % (accuracy_score(y_test, y_pred) * 100))

In [ ]:
# predictions = pd.DataFrame(list(zip(y_test, y_pred)))
# predictions.columns=['actual','prediction']
# predictions['count']=1
# predictions.groupby(['actual','prediction']).count().reset_index().to_csv('most_confusion.csv')

# List Most Relevant Skills

In [24]:
# This code finds the top parameters['num_skills'] of features to show the user. It filters out any 
# ngram where the same n-1 version of the ngram is shown. This cuts down on repetition.

label_id = 21

print(model.classes_[label_id])
print('-------')

features_list = []
topn_class1 = sorted(zip(model.coef_[label_id], vect.get_feature_names()))[-parameters['num_skills']:]
for coef, feat in topn_class1:
    features_list.append(feat)

accepted_skill_list = [model.classes_[label_id]]
for potential_skill in sorted(features_list, key=lambda x: -len(x.split())):
    highest_match = len(potential_skill.split())
    for accepted_skill in accepted_skill_list:
        leftovers = list(set(potential_skill.split()) - set(accepted_skill.split()))
        if len(leftovers) < highest_match:
            highest_match = len(leftovers)
    if highest_match > 1:
        accepted_skill_list.append(potential_skill)
accepted_skill_list = accepted_skill_list[1:]
shuffle(accepted_skill_list)

for skill in accepted_skill_list:
    print(skill)

data scientist
-------
random forest gradient boosting
build predictive model
support vector machine
immersive data science
machine learning python
structure unstructured data
large data set
time series model
data using python
natural language processing nlp
deep neural network
decision tree random forest
principal component analysis
logistic regression model
exploratory data analysis
deep learning model


# Document Similarity Score

In [32]:
# This code returns the prediction probabilities for an example input

example_index = 60
print(y_test[example_index:example_index+1])

print()
print("---------------------")
print(X_test[example_index])

print()
print("---------------------")
vector_example = vect.transform(preprocess_list([X_test[example_index]]))
job_rankings = list(zip(model.predict_proba(vector_example)[0],model.classes_))
sorted(job_rankings,reverse=True)[:20]

168352    financial analyst
Name: cleaned_job_title, dtype: object

---------------------
analyze pay fx oil gas power commodity settlement ach wire forecast analyze funding availability group transaction verify fx rate calculation trader prior executing settlement payment

---------------------


[(0.39948870452995955, 'business analyst'),
 (0.19069868076764576, 'project manager'),
 (0.12289347874090362, 'financial analyst'),
 (0.075204545894490465, 'consultant'),
 (0.070223927627644969, 'software engineer'),
 (0.036880018892284497, 'accountant'),
 (0.019040573440337077, 'product manager'),
 (0.018869533837621397, 'manager'),
 (0.01611220717061216, 'data analyst'),
 (0.014723460746178252, 'engineer'),
 (0.0061903245556760977, 'operations manager'),
 (0.0053895417048063438, 'staff accountant'),
 (0.004965331314362882, 'analyst'),
 (0.0034431225941810857, 'associate'),
 (0.0032647858499209241, 'program manager'),
 (0.0031232562412117628, 'quality assurance analyst'),
 (0.001463640037503908, 'account manager'),
 (0.00091246073392926808, 'quality assurance engineer'),
 (0.00081628788359874383, 'account executive'),
 (0.00061779443167387651, 'systems analyst')]

# Save New Model

In [23]:
# This code saves the model to the models folder

save_time = re.sub('[^A-Za-z0-9]+', '', str(datetime.datetime.now()))
print(save_time)

# write_param = open(directory+"models/" + save_time + '_parameters.txt','w')
# for key in parameters:
#     write_param.write(key + "=" + str(parameters[key]) + '\n')
# write_param.close()

# Save preprocessed x data
pickling_on = open(directory+"models/"+save_time+"_x_data.pkl","wb")
pickle.dump(x_data, pickling_on)
pickling_on.close()

# Save preprocessed y labels
pickling_on = open(directory+"models/"+save_time+"_y_labels.pkl","wb")
pickle.dump(y_labels, pickling_on)
pickling_on.close()

# Save preprocessed x SMOTE data
pickling_on = open(directory+"models/"+save_time+"_x_SMOTE_data.pkl","wb")
pickle.dump(X_res, pickling_on)
pickling_on.close()

# Save preprocessed y SMOTE labels
pickling_on = open(directory+"models/"+save_time+"_y_SMOTE_labels.pkl","wb")
pickle.dump(y_res, pickling_on)
pickling_on.close()

# Save TD-IDF vectorizer
pickling_on = open(directory+"models/"+save_time+"_tdidf_vect.pkl","wb")
pickle.dump(vect, pickling_on)
pickling_on.close()

# # Save vectorized x_train
# pickling_on = open(directory+"models/"+save_time+"_x_trained_tdidf_vect.pkl","wb")
# pickle.dump(X_train_vectorized, pickling_on)
# pickling_on.close()

# # Save NB model
# pickling_on = open(directory+"models/"+save_time+"_nb_model.pkl","wb")
# pickle.dump(model, pickling_on)
# pickling_on.close()

20180718171431719166


OSError: [Errno 22] Invalid argument

# Load Model

In [15]:
# This code loads an old model

save_time = '20180703160501077656' # doc similarity
# save_time = '20180703161959266229' # skills derivation
# save_time = 20180718160301742309 # New model testing SMOTE

pickling_on = open(directory+"models/"+save_time+"_x_data.pkl","rb")
x_data = pickle.load(pickling_on)
pickling_on.close()

# Save preprocessed y labels
pickling_on = open(directory+"models/"+save_time+"_y_labels.pkl","rb")
y_labels = pickle.load(pickling_on)
pickling_on.close()

# Save TD-IDF vectorizer
pickling_on = open(directory+"models/"+save_time+"_tdidf_vect.pkl","rb")
vect = pickle.load(pickling_on)
pickling_on.close()

# Save vectorized x_train
pickling_on = open(directory+"models/"+save_time+"_x_trained_tdidf_vect.pkl","rb")
X_train_vectorized = pickle.load(pickling_on)
pickling_on.close()

# Save NB model
pickling_on = open(directory+"models/"+save_time+"_nb_model.pkl","rb")
model = pickle.load(pickling_on)
pickling_on.close()

/Users/kwheatley/anaconda/envs/python36/lib/python3.6/site-packages/sklearn/base.py:312: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.18.1 when using version 0.19.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


KeyboardInterrupt: 

# End

Most likely I will need to train with a 1-5 ngram model and then return skills
based on a 3-4 ngram model